<a href="https://colab.research.google.com/github/dataskeptic/relatorioPIBIC/blob/main/nilc_wordembeddings_distances_steam_poo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.0/790.0 kB 9.7 MB/s eta 0:00:00


In [2]:
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def to_lowercase(data):
    return data.str.lower()

def remove_punct(data):
    return data.str.replace(r'[^\w\s]', ' ', regex=True)

def remove_stopwords(text):
    stop_words = set(stopwords.words('portuguese'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]

    return ' '.join(filtered_text)

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text, language='portuguese')
    return ' '.join([lemmatizer.lemmatize(token) for token in tokens])

def stem(text):
    stemmer = SnowballStemmer('portuguese')
    tokens = word_tokenize(text, language='portuguese')
    return ' '.join([stemmer.stem(token) for token in tokens])

def preprocess_text(data):
    data = remove_punct(data)
    data = to_lowercase(data)
    data = data.apply(remove_stopwords)
    #data = data.apply(lemmatize)
    data = data.apply(stem)
    return data

In [4]:
path_reference_answers = "drive/MyDrive/PIBIC/data/poo/reference_answers_extended.xlsx"
path_student_answers = "drive/MyDrive/PIBIC/data/poo/student_answers.xlsx"

In [5]:
reference_answers = pd.read_excel(path_reference_answers)
student_answers = pd.read_excel(path_student_answers)

print(reference_answers)
print("---------------------------------------")
print(student_answers)

    question_id                                        refans_text  \
0             1  Polimorfismo é uma característica das linguage...   
1             1  Polimorfismo é a capacidade de uma classe se c...   
2             1  Polimorfismo refere-se à capacidade de um méto...   
3             1  Em programação orientada a objetos, polimorfis...   
4             1  O polimorfismo é um conceito em programação or...   
5             2  Classe são modelos descritivos para a criação ...   
6             2  Uma classe é uma descrição abstrata de um tipo...   
7             2  Classes são estruturas fundamentais em program...   
8             2  Em linguagens orientadas a objetos, uma classe...   
9             2  Classes em programação orientada a objetos ser...   
10            3  Herança é um conceito que permite que uma clas...   
11            3  Herança na programação orientada a objetos é u...   
12            3  Herança é um mecanismo que permite que uma nov...   
13            3  Her

In [6]:
reference_answers['refans_preprocess'] = preprocess_text(reference_answers['refans_text'])

In [7]:
print(reference_answers['refans_preprocess'])

0     polimorf característ linguagens orient objet p...
1     polimorf capac class comport diferent maneir d...
2     polimorf refer capac métod ter vár implement b...
3     program orient objet polimorf princípi class p...
4     polimorf conceit program orient objet refer ca...
5     class model descrit criaçã tip linguag orient ...
6     class descriçã abstrat tip objet defin atribut...
7     class estrutur fundament program orient objet ...
8     linguagens orient objet class model protótip p...
9     class program orient objet serv mold cri objet...
10    heranc conceit permit class absorv estrutur at...
11    heranc program orient objet princípi permit cl...
12    heranc mecan permit nov class herd camp métod ...
13    heranc capac class herd atribut métod outr cla...
14    heranc oop permit class subcl herd característ...
15    privat gt permit acess apen dentr própr classe...
16    exist 4 tip modific acess public tod class pod...
17    jav modific acess determin nível visibil a

In [8]:
student_answers['answer_text'] = student_answers['answer_text'].fillna('')

In [9]:
student_answers['answer_preprocess'] = preprocess_text(student_answers['answer_text'])

In [10]:
def remove_empty(student_answers):
    student_answers = student_answers.apply(lambda x: 'vazio' if pd.isna(x) or x.strip() == '' else x)
    return student_answers

In [11]:
student_answers['answer_preprocess'] = remove_empty(student_answers['answer_preprocess'])

In [12]:
from ot import emd2

/usr/local/lib/python3.10/dist-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


In [13]:
from gensim.models import KeyedVectors
model_cbow = KeyedVectors.load_word2vec_format("drive/MyDrive/PIBIC/wordembeddings/cbow_s300.txt")

In [14]:
model_cbow.init_sims(replace=True)

<ipython-input-14-1f98f1ef571e>:1: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_cbow.init_sims(replace=True)


In [15]:
model_glove = KeyedVectors.load_word2vec_format("drive/MyDrive/PIBIC/wordembeddings/glove_s300.txt")

In [16]:
model_glove.init_sims(replace=True)

<ipython-input-16-ac96e2cc06af>:1: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_glove.init_sims(replace=True)


In [17]:
model_skip = KeyedVectors.load_word2vec_format("drive/MyDrive/PIBIC/wordembeddings/skip_s300.txt")

In [18]:
model_skip.init_sims(replace=True)

<ipython-input-18-c41cc42dd645>:1: DeprecationWarning: Call to deprecated `init_sims` (Use fill_norms() instead. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model_skip.init_sims(replace=True)


In [ ]:
#for index, row in missing_words_df.iterrows():
#    print(f"Index: {index}, Missing Words: {row['missing_words']}")

In [19]:
from gensim.similarities import WmdSimilarity


In [20]:
def compute_wmd_similarities(student_df, reference_df, model, prefix):
    wmd_similarities = pd.DataFrame()

    for questionId in range(1, 5):
        temp_student_answers = student_df[student_df['question_id'] == questionId].copy()
        refans_answers = reference_df['refans_preprocess'][reference_df['question_id'] == questionId].copy()
        refans_answers = [ref.split() for ref in refans_answers]  # Split just once for each questionId
        wmd_similarity = WmdSimilarity(refans_answers, model)

        for index, row in temp_student_answers.iterrows():
            temp_answer = row['answer_preprocess']
            similarities = wmd_similarity[temp_answer.split()]

            similarity = max(similarities), np.mean(similarities), np.median(similarities), min(similarities)
            columns = [f"max_{prefix}", f"mean_{prefix}", f"median_{prefix}", f"min_{prefix}"]
            temp_student_answers.loc[index, columns] = similarity

        wmd_similarities = pd.concat([wmd_similarities, temp_student_answers])

    return wmd_similarities[columns]


final_df = student_answers.copy()

columns_cbow = compute_wmd_similarities(student_answers, reference_answers, model_cbow, "cbow")
final_df = pd.concat([final_df, columns_cbow], axis=1)

columns_glove = compute_wmd_similarities(student_answers, reference_answers, model_glove, "glove")
final_df = pd.concat([final_df, columns_glove], axis=1)

columns_skip = compute_wmd_similarities(student_answers, reference_answers, model_skip, "skip")
final_df = pd.concat([final_df, columns_skip], axis=1)
wmd_similarities = final_df.copy()
print(wmd_similarities)


     question_id                                        answer_text  notas  \
0              1  Polimorfismo é, como o nome sugere (múltiplas ...   1.11   
1              2  Classes são modelos/"estruturas" de coisas tra...   1.11   
2              3  Herança, em POO, é bem fácil de ser entendida ...   1.00   
3              4  Os possíveis modificadores de acesso são 4: pu...   1.11   
4              1  O polimorfismo estático ele permite que nós cr...   0.40   
..           ...                                                ...    ...   
119            4                                            Nao sei   0.00   
120            1  Polimorfismo permite que a partir de um contra...   0.70   
121            2  Classes são estruturas de dados, que reunem um...   0.60   
122            3  Herança permite que atributos ou métodos de um...   1.11   
123            4  public - Permite que qualquer método ou atribu...   0.75   

                                     answer_preprocess  max_cbo

In [21]:
def compute_cosine_similarities(student_df, reference_df, model, prefix):
    cosine_similarities = pd.DataFrame()

    for questionId in range(1, 5):
        temp_student_answers = student_df[student_df['question_id'] == questionId].copy()
        refans_answers = reference_df['refans_preprocess'][reference_df['question_id'] == questionId].copy()
        refans_answers = [ref.split() for ref in refans_answers]  # Split just once for each questionId

        for index, row in temp_student_answers.iterrows():
            #print(f'resposta: {row["answer_preprocess"]}')
            temp_answer = row['answer_preprocess'].split()
            similarities = ([model.n_similarity(temp_answer, ref) for ref in refans_answers])

            similarity = max(similarities), np.mean(similarities), np.median(similarities), min(similarities)
            columns = [f"max_{prefix}", f"mean_{prefix}", f"median_{prefix}", f"min_{prefix}"]
            temp_student_answers.loc[index, columns] = similarity

        cosine_similarities = pd.concat([cosine_similarities, temp_student_answers])

    return cosine_similarities[columns]

# Assuming student_answers and reference_answers are your dataframes and model_cbow, model_glove, and model_skip are your models

final_df = student_answers.copy()

columns_cbow = compute_cosine_similarities(student_answers, reference_answers, model_cbow, "cbow")
final_df = pd.concat([final_df, columns_cbow], axis=1)

columns_glove = compute_cosine_similarities(student_answers, reference_answers, model_glove, "glove")
final_df = pd.concat([final_df, columns_glove], axis=1)

columns_skip = compute_cosine_similarities(student_answers, reference_answers, model_skip, "skip")
final_df = pd.concat([final_df, columns_skip], axis=1)
cosine_similarities = final_df.copy()
print(cosine_similarities)

     question_id                                        answer_text  notas  \
0              1  Polimorfismo é, como o nome sugere (múltiplas ...   1.11   
1              2  Classes são modelos/"estruturas" de coisas tra...   1.11   
2              3  Herança, em POO, é bem fácil de ser entendida ...   1.00   
3              4  Os possíveis modificadores de acesso são 4: pu...   1.11   
4              1  O polimorfismo estático ele permite que nós cr...   0.40   
..           ...                                                ...    ...   
119            4                                            Nao sei   0.00   
120            1  Polimorfismo permite que a partir de um contra...   0.70   
121            2  Classes são estruturas de dados, que reunem um...   0.60   
122            3  Herança permite que atributos ou métodos de um...   1.11   
123            4  public - Permite que qualquer método ou atribu...   0.75   

                                     answer_preprocess  max_cbo

In [22]:
wmd_similarities.describe()

,question_id,notas,max_cbow,mean_cbow,median_cbow,min_cbow,max_glove,mean_glove,median_glove,min_glove,max_skip,mean_skip,median_skip,min_skip
count,124.00000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000
mean,2.50000,0.751774,0.563377,0.531190,0.527799,0.506481,0.561290,0.529225,0.526044,0.504353,0.581479,0.550777,0.547544,0.527966
std,1.12257,0.377058,0.086305,0.054557,0.051652,0.041192,0.087276,0.056679,0.054108,0.043921,0.088168,0.059347,0.056626,0.047585
min,1.00000,0.000000,0.423390,0.422215,0.421986,0.421527,0.415247,0.412465,0.412299,0.410187,0.421209,0.419923,0.419661,0.417941
25%,1.75000,0.600000,0.529082,0.510600,0.508104,0.487616,0.527635,0.508699,0.506996,0.485771,0.547815,0.532639,0.529085,0.514369
50%,2.50000,0.820000,0.560557,0.533687,0.530105,0.509232,0.559740,0.531800,0.528266,0.508159,0.585127,0.557190,0.553198,0.535667
75%,3.25000,1.110000,0.597891,0.562218,0.562360,0.533316,0.594801,0.563030,0.563896,0.533758,0.620565,0.584214,0.585201,0.557485
max,4.00000,1.110000,1.000000,0.698666,0.638922,0.595761,1.000000,0.697204,0.636973,0.595823,1.000000,0.710902,0.654742,0.613857


In [23]:
cosine_similarities.describe()

,question_id,notas,max_cbow,mean_cbow,median_cbow,min_cbow,max_glove,mean_glove,median_glove,min_glove,max_skip,mean_skip,median_skip,min_skip
count,124.00000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000
mean,2.50000,0.751774,0.765741,0.720814,0.724718,0.672986,0.692782,0.619244,0.625264,0.537427,0.817071,0.789818,0.791696,0.760648
std,1.12257,0.377058,0.225577,0.218451,0.219311,0.215322,0.251932,0.252613,0.252498,0.253785,0.263996,0.260937,0.261479,0.258496
min,1.00000,0.000000,0.142179,0.120593,0.114567,0.062869,-0.088804,-0.124401,-0.129093,-0.155560,0.069759,0.058047,0.058720,0.018465
25%,1.75000,0.600000,0.790425,0.751213,0.754276,0.701522,0.675150,0.620637,0.618318,0.516069,0.877122,0.851331,0.855002,0.809122
50%,2.50000,0.820000,0.841065,0.797309,0.798408,0.744425,0.777013,0.706362,0.718253,0.623764,0.912315,0.882804,0.884888,0.851006
75%,3.25000,1.110000,0.882332,0.830609,0.839572,0.778809,0.837199,0.755191,0.768277,0.691890,0.934316,0.905262,0.907637,0.879125
max,4.00000,1.110000,1.000000,0.887577,0.899218,0.858897,1.000000,0.849509,0.867099,0.783959,1.000000,0.939219,0.940072,0.922437


In [24]:
def assign_grades(column):
    median = wmd_similarities[column].median()
    std = wmd_similarities[column].std()
    lower_bound = (median - std)
    upper_bound = (median + std)

    def grade(similarity):
      if similarity < lower_bound:
        return 0
      elif similarity > upper_bound:
          return 3
      else:
          # For values between lower_bound and upper_bound
          distance_to_lower = abs(similarity - lower_bound)
          distance_to_upper = abs(similarity - upper_bound)

          # If similarity is closer to the lower_bound, return 1, otherwise return 2
          return 1 if distance_to_lower < distance_to_upper else 2

    return cosine_similarities[column].apply(grade)

graded_wmd = wmd_similarities.copy()

# Assign grades to each column in the dataframe
for column in wmd_similarities.columns[4:]:
    graded_wmd[f"{column}_grade"] = assign_grades(column)

print(graded_wmd[['max_cbow_grade', 'mean_cbow_grade', 'median_cbow_grade',
       'min_cbow_grade', 'max_glove_grade', 'mean_glove_grade',
       'median_glove_grade', 'min_glove_grade', 'max_skip_grade',
       'mean_skip_grade', 'median_skip_grade', 'min_skip_grade']])

     max_cbow_grade  mean_cbow_grade  median_cbow_grade  min_cbow_grade  \
0                 3                3                  3               3   
1                 3                3                  3               3   
2                 3                3                  3               3   
3                 3                3                  3               3   
4                 3                3                  3               3   
..              ...              ...                ...             ...   
119               0                0                  0               0   
120               3                3                  3               3   
121               3                3                  3               3   
122               3                3                  3               3   
123               3                3                  3               3   

     max_glove_grade  mean_glove_grade  median_glove_grade  min_glove_grade  \
0                  3

In [25]:
def assign_grades(column):
    median = cosine_similarities[column].median()
    std = cosine_similarities[column].std()
    lower_bound = (median - std)
    upper_bound = (median + std)

    def grade(similarity):
      if similarity < lower_bound:
        return 0
      elif similarity > upper_bound:
          return 3
      else:
          # For values between lower_bound and upper_bound
          distance_to_lower = abs(similarity - lower_bound)
          distance_to_upper = abs(similarity - upper_bound)

          # If similarity is closer to the lower_bound, return 1, otherwise return 2
          return 1 if distance_to_lower < distance_to_upper else 2

    return cosine_similarities[column].apply(grade)

graded_cosines = cosine_similarities.copy()

# Assign grades to each column in the dataframe
for column in cosine_similarities.columns[4:]:
    graded_cosines[f"{column}_grade"] = assign_grades(column)

print(graded_cosines[['max_cbow_grade', 'mean_cbow_grade', 'median_cbow_grade',
       'min_cbow_grade', 'max_glove_grade', 'mean_glove_grade',
       'median_glove_grade', 'min_glove_grade', 'max_skip_grade',
       'mean_skip_grade', 'median_skip_grade', 'min_skip_grade']])

     max_cbow_grade  mean_cbow_grade  median_cbow_grade  min_cbow_grade  \
0                 2                2                  2               2   
1                 1                1                  1               1   
2                 2                2                  2               2   
3                 2                2                  2               2   
4                 2                2                  2               2   
..              ...              ...                ...             ...   
119               0                0                  0               0   
120               2                2                  2               2   
121               1                1                  1               1   
122               1                1                  1               2   
123               2                2                  2               2   

     max_glove_grade  mean_glove_grade  median_glove_grade  min_glove_grade  \
0                  2

In [26]:
def similarity_to_score(similarity):
    if similarity <= 0.3 :
        score = 0
    elif similarity <= 0.6:
        score = 1
    elif similarity <= 0.9:
        score = 2
    else:
        score = 3
    return score

graded_cosines['grade'] = graded_cosines['notas'].apply(similarity_to_score)
graded_wmd['grade'] = graded_wmd['notas'].apply(similarity_to_score)

graded_cosines['grade'].value_counts()

3    56
2    38
0    21
1     9
Name: grade, dtype: int64

In [27]:
grade_columns = [col for col in graded_wmd.columns if col.endswith('grade')]

grade_similarities = graded_wmd[grade_columns]

# Show frequencies for each 'grade' column
for col in grade_similarities.columns:
    print(f"Frequencies for {col}:")
    print(graded_wmd[col].value_counts())
    print("\n")

Frequencies for max_cbow_grade:
3    109
0     14
2      1
Name: max_cbow_grade, dtype: int64


Frequencies for mean_cbow_grade:
3    109
0     14
2      1
Name: mean_cbow_grade, dtype: int64


Frequencies for median_cbow_grade:
3    109
0     14
2      1
Name: median_cbow_grade, dtype: int64


Frequencies for min_cbow_grade:
3    109
0     14
2      1
Name: min_cbow_grade, dtype: int64


Frequencies for max_glove_grade:
3    99
0    16
2     7
1     2
Name: max_glove_grade, dtype: int64


Frequencies for mean_glove_grade:
3    97
0    18
2     6
1     3
Name: mean_glove_grade, dtype: int64


Frequencies for median_glove_grade:
3    98
0    18
2     5
1     3
Name: median_glove_grade, dtype: int64


Frequencies for min_glove_grade:
3    82
0    21
2    13
1     8
Name: min_glove_grade, dtype: int64


Frequencies for max_skip_grade:
3    110
0     14
Name: max_skip_grade, dtype: int64


Frequencies for mean_skip_grade:
3    110
0     14
Name: mean_skip_grade, dtype: int64


Frequencies 

In [28]:
grade_columns = [col for col in graded_cosines.columns if col.endswith('grade')]

grade_similarities = graded_cosines[grade_columns]

# Show frequencies for each 'grade' column
for col in grade_similarities.columns:
    print(f"Frequencies for {col}:")
    print(graded_cosines[col].value_counts())
    print("\n")

Frequencies for max_cbow_grade:
2    62
1    47
0    15
Name: max_cbow_grade, dtype: int64


Frequencies for mean_cbow_grade:
2    62
1    47
0    15
Name: mean_cbow_grade, dtype: int64


Frequencies for median_cbow_grade:
2    62
1    47
0    15
Name: median_cbow_grade, dtype: int64


Frequencies for min_cbow_grade:
2    62
1    47
0    15
Name: min_cbow_grade, dtype: int64


Frequencies for max_glove_grade:
2    62
1    45
0    17
Name: max_glove_grade, dtype: int64


Frequencies for mean_glove_grade:
2    62
1    44
0    18
Name: mean_glove_grade, dtype: int64


Frequencies for median_glove_grade:
2    62
1    44
0    18
Name: median_glove_grade, dtype: int64


Frequencies for min_glove_grade:
2    62
1    44
0    18
Name: min_glove_grade, dtype: int64


Frequencies for max_skip_grade:
2    62
1    48
0    14
Name: max_skip_grade, dtype: int64


Frequencies for mean_skip_grade:
2    62
1    48
0    14
Name: mean_skip_grade, dtype: int64


Frequencies for median_skip_grade:
2    62
1

In [29]:
from sklearn.metrics import cohen_kappa_score

In [30]:
similarities_columns = [col for col in graded_wmd.columns if col.endswith('grade') and col != 'grade']
kappa_results = []
# Calculate Cohen's Kappa for each column with the reference 'grade' column
for col in similarities_columns:
    kappa_linear = cohen_kappa_score(graded_wmd[col], graded_wmd['grade'], weights='linear')
    kappa_quadratic = cohen_kappa_score(graded_wmd[col], graded_wmd['grade'], weights='quadratic')

    kappa_results.append({
        'Column': col,
        'Kappa_Linear': kappa_linear,
        'Kappa_Quadratic': kappa_quadratic
    })

# Transformar a lista de dicionários em um DataFrame
kappa_wmd = pd.DataFrame(kappa_results)

print(kappa_wmd)

                Column  Kappa_Linear  Kappa_Quadratic
0       max_cbow_grade      0.432972         0.570687
1      mean_cbow_grade      0.432972         0.570687
2    median_cbow_grade      0.432972         0.570687
3       min_cbow_grade      0.432972         0.570687
4      max_glove_grade      0.545626         0.714987
5     mean_glove_grade      0.561558         0.724964
6   median_glove_grade      0.554680         0.728914
7      min_glove_grade      0.506225         0.666063
8       max_skip_grade      0.425926         0.555753
9      mean_skip_grade      0.425926         0.555753
10   median_skip_grade      0.425926         0.555753
11      min_skip_grade      0.425926         0.555753


In [31]:
similarities_columns = [col for col in graded_cosines.columns if col.endswith('grade') and col != 'grade']
kappa_results = []
# Calculate Cohen's Kappa for each column with the reference 'grade' column
for col in similarities_columns:
    kappa_linear = cohen_kappa_score(graded_cosines[col], graded_cosines['grade'], weights='linear')
    kappa_quadratic = cohen_kappa_score(graded_cosines[col], graded_cosines['grade'], weights='quadratic')

    kappa_results.append({
        'Column': col,
        'Kappa_Linear': kappa_linear,
        'Kappa_Quadratic': kappa_quadratic
    })

# Transformar a lista de dicionários em um DataFrame
kappa_cosine = pd.DataFrame(kappa_results)

print(kappa_cosine)

                Column  Kappa_Linear  Kappa_Quadratic
0       max_cbow_grade      0.288205         0.473704
1      mean_cbow_grade      0.274775         0.443194
2    median_cbow_grade      0.288205         0.450821
3       min_cbow_grade      0.274775         0.435566
4      max_glove_grade      0.321095         0.470882
5     mean_glove_grade      0.304190         0.458467
6   median_glove_grade      0.317443         0.480570
7      min_glove_grade      0.304190         0.458467
8       max_skip_grade      0.278285         0.432715
9      mean_skip_grade      0.291775         0.424997
10   median_skip_grade      0.264795         0.409561
11      min_skip_grade      0.264795         0.409561


In [ ]:
wmd_similarities.to_excel('drive/MyDrive/PIBIC/data/results/similarities/wmd_poo_steam.xlsx', index=False)
#cosine_similarities.to_excel('drive/MyDrive/PIBIC/data/results/similarities/cosine_biology_lemma.xlsx', index=False)
kappa_wmd.to_excel('drive/MyDrive/PIBIC/data/results/kappa/wmd_poo_steam.xlsx', index=False)
#kappa_cosine.to_excel('drive/MyDrive/PIBIC/data/results/kappa/kappa_cosine_biology_lemma.xlsx', index=False)